In [1]:
import re
from pandas import Series, DataFrame
import pandas as pd
import json
import numpy as np
import string

In [2]:
with open('./zhihu_question_seg.txt', 'r' ) as f :
    zhihu_question = [ json.loads( line.strip() ) for line in f]

# Format zh_question.csv

In [3]:
zh_question = DataFrame(zhihu_question)

In [4]:
#zh_question # 70533 rows × 6 columns
ans = zh_question['answer']

In [5]:
qid = []
for i in range( len(ans) ) :
    qid.append(ans[i][0]['url'].split('/')[4])

In [6]:
qid = Series([ str(x) for x in qid ])

In [7]:
zh_question['question_id'] = qid
zh_question = zh_question.drop( zh_question.columns[[0,1,5]], axis = 1)
zh_question.to_csv('./zh_question.csv', encoding='utf-8', index = False )

# Format zh_answer.csv

In [8]:
zhihu_question = DataFrame(zhihu_question)
zhihu_answer = zhihu_question['answer']

In [9]:
zhihu_answer.to_csv('./zhihu_answer.csv', encoding='utf-8', index = False)

In [10]:
with open('./zhihu_answer.csv', 'r') as f_in, \
    open('./zhihu_answer_pro.csv', 'w') as f_out :
    # line = f.readline()
    
    for line in f_in :
        answers = line.split('},') 
        for answer in answers: 
            
            answer = answer.strip().strip('"').strip('[').strip(']')
            answer = answer.replace('"', '')
            answer = answer.replace('{u\'','{\'')
            answer = answer.replace(': u\'', ': \'')
            answer = answer.replace(', u\'', ', \'')
            answer = answer.replace('\'', '"')
            
            if answer.endswith('}') :
                f_out.write(answer)
                f_out.write('\n')
            else :
                f_out.write(answer)
                f_out.write('}\n')


In [11]:
zh_ans = []

In [12]:
with open('./zhihu_answer_pro.csv', 'r') as f:
    for i in range(172932) :
    # for i in range(10) :
        # print i
        line = f.readline()
        try :
            zh_ans.append( json.loads(line) )
        except ValueError :
            # print i
            continue

In [13]:
zh_ans = DataFrame(zh_ans)

In [14]:
# zh_ans # 171656 rows × 5 columns

In [15]:
question_id = zh_ans['url'].str.split('/').str[4]
answer_id = zh_ans['url'].str.split('/').str[-1]
answerer_id = zh_ans['answerer'].str.split('/').str[-1]

In [16]:
zh_ans['question_id'] = question_id
zh_ans['answer_id'] = answer_id
zh_ans['answerer_id'] = answerer_id
zh_ans = zh_ans.drop( zh_ans.columns[[0,2,4]], axis=1)
zh_ans.to_csv('./zh_answer.csv', encoding='utf-8', index = False )

# Format zh_user.csv

In [20]:
with open('./zhihu_user.txt', 'r' ) as f :
    zhihu_user = [ json.loads( line.strip() ) for line in f]

In [21]:
zh_user = DataFrame(zhihu_user)

In [22]:
answerer_id = zh_user['url'].str.split('/').str[-1]

In [23]:
# answerer_id

In [24]:
zh_user['answerer_id'] = answerer_id

In [25]:
zh_user = zh_user.drop('url', axis=1)

In [26]:
# zh_user  # 71856 rows × 11 columns

In [27]:
zh_user.to_csv('./zh_user.csv', encoding='utf-8', index = False )

# Merge quetion & answer & user

In [29]:
question = pd.read_csv('./zh_question.csv')
answer = pd.read_csv('./zh_answer.csv')
user = pd.read_csv('./zh_user.csv')

In [30]:
question = question.drop_duplicates('question_id')
answer = answer.drop_duplicates('answer_id')
user = user.drop_duplicates('answerer_id')

In [31]:
# question # 69746 rows × 4 columns
# answer   # 168112 rows × 5 columns
# user     # 71856 rows × 11 columns

In [32]:
# m = pd.merge(m1, m2, on = 'question_id')
answer_user = pd.merge(answer, user, on = 'answerer_id', how = 'left')

In [33]:
# answer_user # 168112 rows × 15 columns

In [34]:
question_answer_user = pd.merge(question, answer_user, on = 'question_id', how = 'right')

In [35]:
# question_answer_user # 168112 rows × 18 columns

In [36]:
question_answer_user.to_csv('./zh_data.csv', encoding='utf-8', index = False )